# Project Idea: Text-to-Phoneme Conversion

**Objective**: A critical step in many TTS systems is converting the text into phonemes (distinct units of sound). This project involve building a model that takes standard English text and predicts its phonemic transcription in X-SAMPA format.

**Steps**:

1. **Data Preparation**:
    - Split CSV file into training, validation, and test sets.

2. **Preprocessing**:
    - Character tokenization the English text
    - Tokenize the X-SAMPA transcriptions. Here, each unique X-SAMPA symbol can be a token.
    - Convert these tokens to integer representations.

3. **Modeling**:
    - You can use sequence-to-sequence models, which have been successful for such tasks. LSTM or GRU based architectures would be a good starting point.
    - Input is the tokenized English text, and the target is the X-SAMPA transcription.

4. **Training & Evaluation**:
    - Train your model on the training set, evaluate on the validation set, and fine-tune accordingly.
    - Once satisfied, evaluate its performance on the test set.

5. **Inference**:
    - Build a function where you input standard English text, and it outputs the predicted X-SAMPA transcription.

**Technologies**:
- Python
    - ``csv``
    - ``keras``
    - ``sklearn``
- X-Sampa
- librosa (for audio processing)

#### 1.1 Split CSV file into training, validation, and test sets.

In [11]:
import pandas as pd

# Load the CSV data
data = pd.read_csv('transcription.csv')

from sklearn.model_selection import train_test_split

# Group by WordID
grouped = list(data.groupby('WordID'))

# Split the grouped data
train, temp = train_test_split(grouped, test_size=0.3, random_state=42)  
valid, test = train_test_split(temp, test_size=0.5, random_state=42)  

# Extracting the dataframes from the groups
train_df = pd.concat([item[1] for item in train])
valid_df = pd.concat([item[1] for item in valid])
test_df = pd.concat([item[1] for item in test])

# X-SAMPA tokenization
def tokenize_x_sampa(transcription):
    # List of known multi-character X-SAMPA symbols 
    # This list might need more symbols based on the entire dataset
    multi_char_symbols = [":l", "^m", "O:"]
    
    tokens = []
    for symbol in transcription.split():
        if symbol in multi_char_symbols:
            tokens.append(symbol)
        else:
            tokens.extend(list(symbol))
    return tokens

# Apply the function to the Transcription column for each dataframe
train_df['tokenized_x_sampa'] = train_df['Transcription'].apply(tokenize_x_sampa)
valid_df['tokenized_x_sampa'] = valid_df['Transcription'].apply(tokenize_x_sampa)
test_df['tokenized_x_sampa'] = test_df['Transcription'].apply(tokenize_x_sampa)

# Display the first few rows for verification
print(train_df[['Transcription', 'tokenized_x_sampa']].head())

    Transcription                     tokenized_x_sampa
34  fAr h@"raIz@n  [f, A, r, h, @, ", r, a, I, z, @, n]
35  fA: h@"raIz@n  [f, A, :, h, @, ", r, a, I, z, @, n]
92  Endl@s oUS@nz  [E, n, d, l, @, s, o, U, S, @, n, z]
93  EndlIs oUS@nz  [E, n, d, l, I, s, o, U, S, @, n, z]
98   glIs.nIN du:     [g, l, I, s, ., n, I, N, d, u, :]


#### 2.1 Character tokenization the English text

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# Function to convert the list of tokens back to space-separated string
def tokens_to_string(tokens):
    return ' '.join(tokens)

# Convert the list of tokens in 'tokenized_x_sampa' back to space-separated string
train_df['x_sampa_str'] = train_df['tokenized_x_sampa'].apply(tokens_to_string)

# Initialize a CountVectorizer
vectorizer = CountVectorizer(analyzer='word', token_pattern=r"(?:\S+)")
# We're using the token_pattern parameter to make sure we capture the entire X-SAMPA symbols

# Apply the vectorizer on the 'x_sampa_str' column
X = vectorizer.fit_transform(train_df['x_sampa_str'])

# Getting the token names
tokens = vectorizer.get_feature_names()

print("Tokens (X-SAMPA symbols):", tokens)



Tokens (X-SAMPA symbols): ['"', '.', '3', ':', '@', '^', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'z', 'ð', 'ŋ']


#### 2.2 Tokenize the X-SAMPA transcriptions.

In [14]:
# List of known multi-character X-SAMPA symbols (this is just a sample; there might be more)
multi_char_symbols = [":l", "^m", "O:"]
# If there are more multi-character symbols in your data, add them to the list

def tokenize_x_sampa(transcription):
    tokens = []
    i = 0
    while i < len(transcription):
        # Check if the current char and next char form a multi-char symbol
        if i < len(transcription) - 1 and transcription[i:i+2] in multi_char_symbols:
            tokens.append(transcription[i:i+2])
            i += 2
        # Check for single-char symbols
        elif transcription[i] != ' ':
            tokens.append(transcription[i])
            i += 1
        # If it's just a space, skip to the next character
        else:
            i += 1
    return tokens

# Apply the function to the Transcription column of train_df
train_df['tokenized_x_sampa'] = train_df['Transcription'].apply(tokenize_x_sampa)

# Display the results for train_df
print(train_df[['Transcription', 'tokenized_x_sampa']])



      Transcription                      tokenized_x_sampa
34    fAr h@"raIz@n   [f, A, r, h, @, ", r, a, I, z, @, n]
35    fA: h@"raIz@n   [f, A, :, h, @, ", r, a, I, z, @, n]
92    Endl@s oUS@nz   [E, n, d, l, @, s, o, U, S, @, n, z]
93    EndlIs oUS@nz   [E, n, d, l, I, s, o, U, S, @, n, z]
98     glIs.nIN du:      [g, l, I, s, ., n, I, N, d, u, :]
..              ...                                    ...
153  wO:m "s^nlaIt"  [w, O:, m, ", s, ^, n, l, a, I, t, "]
70       raIzIN s^n            [r, a, I, z, I, N, s, ^, n]
71       raIzIN s^n            [r, a, I, z, I, N, s, ^, n]
172   sAft "pIloUz"   [s, A, f, t, ", p, I, l, o, U, z, "]
173   sOft "pIl@Uz"   [s, O, f, t, ", p, I, l, @, U, z, "]

[160 rows x 2 columns]


#### 2.3 Convert these tokens to integer representations.

In [15]:
from keras.preprocessing.text import Tokenizer

# Convert the list of tokens back to space-separated strings for both English and X-SAMPA
train_df['english_str'] = train_df['Word'].apply(lambda x: ' '.join(list(x)))  # For English character tokenization
train_df['x_sampa_str'] = train_df['tokenized_x_sampa'].apply(' '.join)

valid_df['english_str'] = valid_df['Word'].apply(lambda x: ' '.join(list(x)))
valid_df['x_sampa_str'] = valid_df['tokenized_x_sampa'].apply(' '.join)

test_df['english_str'] = test_df['Word'].apply(lambda x: ' '.join(list(x)))
test_df['x_sampa_str'] = test_df['tokenized_x_sampa'].apply(' '.join)

# Initialize and fit the tokenizer for English text
english_tokenizer = Tokenizer(char_level=False, lower=True, split=' ')
english_tokenizer.fit_on_texts(train_df['english_str'])

# Convert English tokens to integer sequences
train_df['english_seq'] = english_tokenizer.texts_to_sequences(train_df['english_str'])
valid_df['english_seq'] = english_tokenizer.texts_to_sequences(valid_df['english_str'])
test_df['english_seq'] = english_tokenizer.texts_to_sequences(test_df['english_str'])

# Initialize and fit the tokenizer for X-SAMPA transcriptions
x_sampa_tokenizer = Tokenizer(char_level=False, lower=False, split=' ')
x_sampa_tokenizer.fit_on_texts(train_df['x_sampa_str'])

# Convert X-SAMPA tokens to integer sequences
train_df['x_sampa_seq'] = x_sampa_tokenizer.texts_to_sequences(train_df['x_sampa_str'])
valid_df['x_sampa_seq'] = x_sampa_tokenizer.texts_to_sequences(valid_df['x_sampa_str'])
test_df['x_sampa_seq'] = x_sampa_tokenizer.texts_to_sequences(test_df['x_sampa_str'])

# Let's check the conversion
print(train_df[['english_str', 'english_seq', 'x_sampa_str', 'x_sampa_seq']].head())


                    english_str                                 english_seq  \
34        F a r   h o r i z o n         [19, 8, 5, 11, 10, 5, 2, 23, 10, 4]   
35        F a r   h o r i z o n         [19, 8, 5, 11, 10, 5, 2, 23, 10, 4]   
92  E n d l e s s   o c e a n s  [3, 4, 12, 6, 3, 1, 1, 10, 16, 3, 8, 4, 1]   
93  E n d l e s s   o c e a n s  [3, 4, 12, 6, 3, 1, 1, 10, 16, 3, 8, 4, 1]   
98  G l i s t e n i n g   d e w   [7, 6, 2, 1, 9, 3, 4, 2, 4, 7, 12, 3, 13]   

                x_sampa_str                            x_sampa_seq  
34  f A r h @ " r a I z @ n         [21, 14, 5, 25, 5, 9, 1, 7, 3]  
35  f A : h @ " r a I z @ n            [21, 14, 25, 5, 9, 1, 7, 3]  
92  E n d l @ s o U S @ n z     [26, 3, 8, 4, 2, 24, 12, 18, 3, 7]  
93  E n d l I s o U S @ n z  [26, 3, 8, 4, 1, 2, 24, 12, 18, 3, 7]  
98    g l I s . n I N d u :         [19, 4, 1, 2, 3, 1, 10, 8, 29]  


In [ ]:
#### 3.1 Split CSV file into training, validation, and test sets.

In [ ]:
#### 3.1 Split CSV file into training, validation, and test sets.

In [ ]:
#### 4.1 Split CSV file into training, validation, and test sets.

In [ ]:
#### 4.2 Split CSV file into training, validation, and test sets.

In [ ]:
#### 5.1 Split CSV file into training, validation, and test sets.